## Lab5c: Help Session of Assignment 5 

#### Introduction

This lab aims to help you start on assignment 5, which is a real recommendation system build on top of MapReduce framework. We will go through each steps of workflow in assignment 5 and show result of each mapper and reducer functions. You can use these results for debugging.

#### Tiny Data Result



## Assignment 5: Recommendation System with Big Data

## Overview
I LOVE MOVIE corp has been more and more popular since their last collaboration with you. To make more money, just like all the other companies, they plan to build a movie recommendation system for their customers. However, because they are too popular, well, and have plenty of money, they decide to use Big Data engines and build their system on Google Cloud. As their technology counselor, you will fulfill this requirement in this assginment.

** OBJECTIVE **:

In this assignment, you will implement a movie recommendation system based on the [US patent](https://docs.google.com/viewer?url=patentimages.storage.googleapis.com/pdfs/US7908183.pdf) of Amazon.com recommendation system. You will use [item-to-item](https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf) algorithm to compute item similarity. All the algorithms will be implement in MapReduce framework we have used in Lab1, while using MongoDB to query the result.

** TASKS **

1. Process batch data with MapReduce
1. Use NoSQL to interact with MongoDB

** ATTENTION **: Lab 5c shows the result of each steps of this code on a small dataset, as long as detailed explanation of each steps. Refer to it when you get stucked. 

### Part 1: Build Movie Recommendation System
In this part, you will achieve a movie recommenation system based on Amazon.com recommendation system. Below is the workflows that we will implement. Figure 3 shows the algorithm for batch. The following instruction will help you understand each steps of this workflow, and how you should implement them in MapReduce. We will use the number beside each rectangle to denote steps. Feel free to use code we provided before, they are really useful.

 <img src="workflow.jpeg" width = "600" height = "800" alt="what" align=center />

Here we provide you two functions that may be useful. `Jaccard_similarity` calculates [Jaccard similarity](http://en.wikipedia.org/wiki/Jaccard_index) values, with `n_common` as overlapped users and `n1` and `n2` as user numbers for each movie. `combinations` return the list of all possible combinations for r items in iterable. Refer to document [here](http://docs.python.org/2/library/itertools.html#itertools.combinations).

** NOTE **: 

- step 102 and 106 are not needed in our system. In other word, you don't need to implement them.
- we define 4 global variables at the beginning of our program. These variables will be used in your mapper functions. DO NOT MODIFY THEM! Otherwise we couldn't grade your assignment..

** STEP 100 **: In our case, 'purchase history' refers to user ratings for each movie. Use pandas to read in `ratings.csv`, and convert records to list using dataframe [.values](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.values.html) attribute and `tolist()` function. the output should looks like:
~~~~
[[1,2,3.5],
 [3,4,4.0], ...]
~~~~

For mapreduce part, we already offered you the main function and workflow of how to call each mapper and reducer function. DO NOT MODIFY IT! What you need to do here is fill in mapper functions based on our instruction below. Note that we already filled in `reducer` function, cause it is exactly what you did in lab5.1.

** STEP 104**: Fill in function `mapper1` to Map purchased items to customers. The input and output format of this function should be:
~~~~
:param record: "user_id, movie_id, rating, timestamp"
:return: (key, value)
          key: movie_id
          value: [(user_id, rating)]
~~~~
As you can see, input record is just each row we have read from csv file. The output if user_id, rating tuple wrapped by list. Recall that we do this because of flatMap requirement, and it's actually helps us to reduce them by just using operator '+'.

** STEP 108 **

This one is a little bit tricky. You need to complete both `mapper2` and `mapper3` for this step. `mapper2` simply generates key value pair where user_id is the key and all the other information as value. Reduce it by key, we now get the list for all movies that a user has rated. `mapper3` generates tuples of each popular movie-other movie pair, with  number of rating users for each movie as its value. Reduce it by key, now each item in value list corresponds to one common user who rates both movies in key tuple.

** note **: In function `mapper3`, you need to filter movie pairs whose `movie_id1` is in `popular_item` list. We already provided you `popular_item` list.
~~~~
mapper2(record):
    input: (key, value)
            key: movie_id
            value: list of (user_id, rating) tuples
    output: list of (key, value) pair
            key: user_id
            value: [(movie_id, rating, number of rater)]
mapper3(record):
    input: (key, value)
            key: user_id
            value: list of (movie_id, rating, number of rater)
    output: list of (key, value) tuple
            key: (movie_id1, movie_id2)
            value: ([(num_rater1, num_rater2)])
~~~~
Although we don't contain the number of common user explicitly in the output, the length of value list is actually equal to common user number, which will be used in later step.

** STEP 110 & 114 **

We will flip 112 and 114 steps. Our algorithm uses Jaccard similarity to measure similarity between movies and only returns movie pairs that share **more than** RATING_NUM (=3) common users. Fill in function `mapper4` to calculate Jaccard similarity. Recall that the length of value list is COMMON USER NUMBER!
~~~~
mapper4(record):
    input: (key, value)
            key: (movie_id1, movie_id2)
            value: list of (num_rater1, num_rater2) pair, where length of this list equal to number of common users
    output: [(key, value)] or []
            key: movie_id
            value: [(movie_id2, jaccard)]
            if they doesn't satisfy RATING_NUM constrain, return []
~~~~

** 112 & 116 **

Now, we need to sort other item lists for each popular item and truncate this list to n (=3), which is a global variable defined at the top of our code.  Fill in function `mapper5` for it.
~~~~
mapper5(record):
    input: (key, value)
            key: movie_id1
            value: [(movie_id2, jaccard), ...]
    output:(movie_id1, [(movie_id, jaccard), (movie_id, jaccard), ...])
            sorted tuples with n other items
~~~~

** after 116 **

Until now, you have achieved all the steps in batch workflow. we now need to upload this similar table into MongoDB. Use the same write function in step 100, and name your collection similar. Make sure each document in your collection has the following schema:
~~~~
{'_id': ObjectId, 'popular_id': int, 'item_list': [{'_1': int, '_2': float}, ..., 'item_n':{...}]}
~~~~
Now you can check your result with the following NoSQL. The result should be the same with problem 3 at NoSQL lab 3.


### Part 2: Sparse Matrix Multiplication

In [ ]:
from mapreduce import mapreduce
from pprint import pprint
import string

i_range = 5
j_range = 5
import json
from mapreduce import mapreduce
def mapper1(record):
    """
    :param record: (matrix, i, j, value)
                    matrix: which matrixId this record belong to
                    i: row number
                    j: column number
                    value: value
    return:        [((k, f), record), ...]
                    k: the corresponding new matrix row indexs
                    f: the corresponding new matrix column indexs
                    record: input record
    """
    # YOUR CODE HERE
    
# same as task 1
def reducer(a, b):
    
    # YOUR CODE HERE

def mapper2(record):
    """
    :param record: ((k, f), [(matrix, i, j, value),...])
    :return         (k, f, value)
                    value: new matrix value at position (k, f)
    """
    # YOUR CODE HERE

def matrix_multiplication():
    '''
    Multiplies two sparse matrices A and B where A and B are represented as lists of tuples of the 
    form (i,j, value).  The product is returned using this same format.'''
    # don't modify
    with open('data/matrix.json', 'r') as infile:
        data = [json.loads(line) for line in infile]

    sc = mapreduce()
    multiply_result = sc.parallelize(data, 128) \
        .flatMap(mapper1) \
        .reduceByKey(reducer) \
        .map(mapper2) \
        .collect()
    sc.stop()
    pprint(multiply_result.sort())
    
matrix_multiplication()